## 0. Cargando la Data y Packages

In [1]:
using Pkg
Pkg.add("CSV")
Pkg.add("RData")
Pkg.add("DataFrames")
Pkg.add("StatsBase")
Pkg.add("StatsModels")
Pkg.add("Statistics")
Pkg.add("GLM")
Pkg.add("LaTeXStrings")
Pkg.add("CategoricalArrays")


    Updating registry at `C:\Users\abrah\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No

In [88]:
using CSV
using GLM
using DataFrames
using LaTeXStrings
using StatsBase
using Statistics,RData 
using CategoricalArrays

#podemos cargar esto como csv
df_csv = CSV.read("../../../data/wage2015_subsample_inference.csv", DataFrame)
#pero también como RData
#df_rdata = load("../../../data/wage2015_subsample_inference.RData")["data"]

#podemos usar el archivo RData sobretodo porque detecta Categoricals. Pero esta vez usaremos  el archivo csv
df = df_csv
#para ver las primeras 5 observacione

first(df, 5) 


Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22


## 1. Data analysis

#### 2. Are there missing values? Display the number of missings (if any) of each variable in the data set.


#### 3. Report descriptive statistics of the variables (mean, standard deviation, percentiles, etc.). Interpret your results.

In [89]:
# Usamos describe para obtener algunos estadísticos descriptivos y ver los missingvalues de caca variable del data set
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,rownames,15636.3,10,15260.0,32643,0,Int64
2,wage,23.4104,3.02198,19.2308,528.846,0,Float64
3,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
4,sex,0.444466,0.0,0.0,1.0,0,Float64
5,shs,0.023301,0.0,0.0,1.0,0,Float64
6,hsg,0.243883,0.0,0.0,1.0,0,Float64
7,scl,0.278058,0.0,0.0,1.0,0,Float64
8,clg,0.31767,0.0,0.0,1.0,0,Float64
9,ad,0.137087,0.0,0.0,1.0,0,Float64


In [19]:
using Statistics
#encontramos algunos los cuartiles de la variable wage:
q1 = quantile(df.wage, 0.25)  
q2 = quantile(df.wage, 0.50)  
q3 = quantile(df.wage, 0.75)  

#imprimimos los valores de cada cuartil
println("Q1: $q1")
println("Mediana: $q2")
println("Q3: $q3")


Q1: 13.461538461538462
Mediana: 19.23076923076923
Q3: 27.77777777777778


### 4. How many women with a college graduate degree (clg) or above have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result

In [20]:
#nos están preguntando por cuantas muejeres graduadas de universidad o con postgrado se encuentran el cuartil superior de salarios. (mayores a 27.7777)

df[(df.sex .== 1.0) .& ((df.clg .== 1.0) .|| (df.ad .== 1.0)) .& (df.wage .>= q3), :]

Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,19,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
2,191,42.3077,3.74497,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4700.0,16,4795.0,9
3,232,41.2088,3.71865,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,11.0,1.21,1.331,1.4641,4700.0,16,5790.0,9
4,319,100.0,4.60517,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,27.0,7.29,19.683,53.1441,850.0,2,6970.0,12
5,563,33.6538,3.51613,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,0.16,0.064,0.0256,710.0,2,7390.0,14
6,842,36.0577,3.58512,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,23.0,5.29,12.167,27.9841,420.0,1,9170.0,21
7,865,31.25,3.44202,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,15.0,2.25,3.375,5.0625,3255.0,10,8190.0,18
8,892,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9.0,0.81,0.729,0.6561,540.0,2,6990.0,12
9,1052,48.0769,3.8728,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,24.0,5.76,13.824,33.1776,4850.0,16,3390.0,6


In [21]:
#para saber el número de mujeres que cumplen con estos requisitos:
mujeres_with_college_or_superior_degree_in_q3=nrow(df[(df.sex .== 1.0) .& ((df.clg .== 1.0) .|| (df.ad .== 1.0)) .& (df.wage .>= q3), :])

println("Las mujeres con universidad o un grado superior en el cuartil superior de salarios son: $mujeres_with_college_or_superior_degree_in_q3")

Las mujeres con universidad o un grado superior en el cuartil superior de salarios son: 419


#### 5. How many men with a high school graduate degree (hsg) or below have a wage corresponding to the 25% richest of the sample? Report the dataframe corresponding to this conditions and the result.

In [22]:
#hacemos algo similar para los hombres:
men_with_highschool_or_no_degree_in_q3=nrow(df[(df.sex .== 0.0) .& ((df.shs .== 1.0) .|| (df.hsg .== 1.0)) .& (df.wage .>= q3), :])

println("Los hombres con high school terminada o un grado menor en el cuartil superior de salarios son: $men_with_highschool_or_no_degree_in_q3")

Los hombres con high school terminada o un grado menor en el cuartil superior de salarios son: 118


#### 6. Create two dataframes. One containing only the log(wage) and the other containig every variable of the data set but the wage related variables.

In [72]:
#creamos los 2 dataframes

#1er dataframe (only log(wage))

only_logwage = DataFrame(log_wage = df.lwage)

#2do dataframe (otras variables)

df_no_wage = select(df, Not([:wage, :lwage]))



Row,rownames,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,30,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,43,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,44,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,47,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


## 2. Data wrangling

#### 7. Make an array for our Y variable, which will be the logarithm of wage (lwage column)

In [73]:
#convierto en array con nombre Y
Y=df.lwage

5150-element Vector{Float64}:
 2.2633643798407643
 3.872802292274865
 2.403126322215923
 2.634927936273247
 3.361976668508874
 2.4622152385859297
 2.9565115604007097
 2.9565115604007097
 2.4849066497880004
 2.9565115604007097
 ⋮
 3.117779707996832
 2.822980167776187
 3.1796551117149194
 2.6280074934286737
 2.6925460145662448
 3.138833117194664
 3.649658740960655
 3.4955080611333966
 2.8511510447428834

In [74]:
#primero convertimos en matriz  y quitamos las variables que no vamos a usar:
no_wage = Matrix(select(df, Not([:wage, :lwage, :occ, :ind],)))


5150×17 Matrix{Float64}:
    10.0  1.0  0.0  0.0  0.0  1.0  …   0.49   0.343    0.2401  11.0  18.0
    12.0  0.0  0.0  0.0  0.0  1.0      9.61  29.791   92.3521  10.0   9.0
    15.0  0.0  0.0  1.0  0.0  0.0      3.24   5.832   10.4976  19.0   4.0
    18.0  1.0  0.0  0.0  0.0  0.0      6.25  15.625   39.0625   1.0  12.0
    19.0  1.0  0.0  0.0  0.0  1.0      4.84  10.648   23.4256   6.0  22.0
    30.0  1.0  0.0  0.0  0.0  1.0  …   0.01   0.001    0.0001   5.0  14.0
    43.0  1.0  0.0  1.0  0.0  0.0     17.64  74.088  311.17    17.0  14.0
    44.0  0.0  0.0  1.0  0.0  0.0     13.69  50.653  187.416   17.0   9.0
    47.0  1.0  0.0  1.0  0.0  0.0      9.61  29.791   92.3521  13.0  19.0
    71.0  1.0  0.0  0.0  0.0  1.0      0.16   0.064    0.0256  10.0  18.0
     ⋮                        ⋮    ⋱                            ⋮    
 32599.0  0.0  0.0  1.0  0.0  0.0      2.25   3.375    5.0625  22.0   9.0
 32603.0  0.0  0.0  1.0  0.0  0.0      1.21   1.331    1.4641  20.0  21.0
 32606.0  1.0  0.

In [90]:
#filtramos y nos quedamos con las variables que nos contengan wage y que nos interesen
df=select!(df, Not([:rownames, :occ, :ind, :wage]))

first(df_no_wage, 5)

Row,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ2,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,11,18
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,10,9
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,19,4
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,1,12
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,6,22


In [27]:
#vemos los valores que toma la variable occ2 para luego convertirla en dummie
sort(unique(df_no_wage.occ2))
#efectivamente son valores del 1 al 22. Por tanto podemos crear una dummie tomando como base a 1

22-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
  ⋮
 14
 15
 16
 17
 18
 19
 20
 21
 22

In [98]:
#convertimos a occ2 en dummie:
for i in 1:22
    col_name = Symbol("occ2_$i")
    df = transform(df, :occ2 => ByRow(x -> x == i ? 1 : 0) => col_name)
end

ncol(df)

60

In [77]:
#vemos los valores que toma la variable ind2 para luego convertirla en dummie
sort(unique(df_no_wage.ind2))
#efectivamente son valores del 2 al 22. Por tanto podemos crear una dummie tomando como base a

21-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
  ⋮
 14
 15
 16
 17
 18
 19
 20
 21
 22

In [97]:
#convertimos ind2 en dummie:
for i in 2:22
    col_name = Symbol("ind2_$i")
    df = transform(df, :occ2 => ByRow(x -> x == i ? 1 : 0) => col_name)
end
df

Row,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ2,ind2,occ2_1,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,ind2_22
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,11,18,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,10,9,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,19,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,1,12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,6,22,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,5,14,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,17,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,17,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,13,19,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [82]:
#eliminando variables que pueden causar multicolinealidad. También rownames y las occ2 y ind2 que se convirtieron en dummies
df=select!(df, Not([:occ2, :ind2, :ad, :ne, :occ2_1, :ind2_2]))
df

ArgumentError: ArgumentError: column name :rownames not found in the data frame

In [81]:
ncol(df)

55

In [69]:
# Usar transform para crear una nueva columna
exp_vars = Symbol.("exp", 1:4)
other_vars = vcat(Symbol.("shs"), Symbol.("hsg"), Symbol.("clg"), Symbol.("mw"), Symbol.("so"), Symbol.("we"), Symbol.("occ2_", 2:22),Symbol.("ind2_", 3:22))

# Usar un bucle for para crear las nuevas columnas
for exp_var in exp_vars
    for other_var in other_vars
        col_name = Symbol("$(exp_var)_$(other_var)")
        df_nuevo = transform(df_nuevo, [exp_var, other_var] => ((x, y) -> x .* y) => col_name)
    end
end
# Modificar una columna existente
#df = transform(df, :Var1 => x -> x .* 2 => :Var1)
df_nuevo

Row,wage,lwage,sex,shs,hsg,scl,clg,mw,so,we,exp1,exp2,exp3,exp4,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,ind2_22,exp1_shs,exp1_hsg,exp1_clg,exp1_mw,exp1_so,exp1_we,exp1_occ2_2,exp1_occ2_3,exp1_occ2_4,exp1_occ2_5,exp1_occ2_6,exp1_occ2_7,exp1_occ2_8,exp1_occ2_9,exp1_occ2_10,exp1_occ2_11,exp1_occ2_12,exp1_occ2_13,exp1_occ2_14,exp1_occ2_15,exp1_occ2_16,exp1_occ2_17,exp1_occ2_18,exp1_occ2_19,exp1_occ2_20,exp1_occ2_21,exp1_occ2_22,exp2_shs,exp2_hsg,exp2_clg,exp2_mw,exp2_so,exp2_we,exp2_occ2_2,exp2_occ2_3,exp2_occ2_4,exp2_occ2_5,exp2_occ2_6,exp2_occ2_7,exp2_occ2_8,exp2_occ2_9,exp2_occ2_10,exp2_occ2_11,exp2_occ2_12,exp2_occ2_13,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,0.49,0.343,0.2401,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.49,0.0,0.0,⋯
2,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,31.0,9.61,29.791,92.3521,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.61,0.0,0.0,0.0,⋯
3,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,18.0,3.24,5.832,10.4976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,3.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,6.25,15.625,39.0625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,4.84,10.648,23.4256,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.01,0.001,0.0001,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
7,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,42.0,17.64,74.088,311.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,17.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
8

In [67]:
ncol(df_others)

UndefVarError: UndefVarError: `df_others` not defined

In [63]:
# Usar transform para crear una nueva columna
vars = vcat(Symbol.("exp", 1:4),Symbol.("shs"), Symbol.("hsg"), Symbol.("clg"), Symbol.("mw"), Symbol.("so"), Symbol.("we"), Symbol.("occ2_", 2:22),Symbol.("ind2_", 3:22))
#other_vars = vcat(Symbol.("shs"), Symbol.("hsg"), Symbol.("clg"), Symbol.("mw"), Symbol.("so"), Symbol.("we"), Symbol.("occ2_", 2:22))

# Usar un bucle for para crear las nuevas columnas
for var in vars
    for va in vars
        col_name = Symbol("$(var)_$(var)")
        df_nuevo = transform(df_nuevo, [var, var] => ((x, y) -> x .* y) => col_name)
    end
end
# Modificar una columna existente
#df = transform(df, :Var1 => x -> x .* 2 => :Var1)
df_nuevo

Row,wage,lwage,sex,shs,hsg,scl,clg,mw,so,we,exp1,exp2,exp3,exp4,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,ind2_22,exp1_shs,exp1_hsg,exp1_clg,exp1_mw,exp1_so,exp1_we,exp1_occ2_2,exp1_occ2_3,exp1_occ2_4,exp1_occ2_5,exp1_occ2_6,exp1_occ2_7,exp1_occ2_8,exp1_occ2_9,exp1_occ2_10,exp1_occ2_11,exp1_occ2_12,exp1_occ2_13,exp1_occ2_14,exp1_occ2_15,exp1_occ2_16,exp1_occ2_17,exp1_occ2_18,exp1_occ2_19,exp1_occ2_20,exp1_occ2_21,exp1_occ2_22,exp2_shs,exp2_hsg,exp2_clg,exp2_mw,exp2_so,exp2_we,exp2_occ2_2,exp2_occ2_3,exp2_occ2_4,exp2_occ2_5,exp2_occ2_6,exp2_occ2_7,exp2_occ2_8,exp2_occ2_9,exp2_occ2_10,exp2_occ2_11,exp2_occ2_12,exp2_occ2_13,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,0.49,0.343,0.2401,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.49,0.0,0.0,⋯
2,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,31.0,9.61,29.791,92.3521,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.61,0.0,0.0,0.0,⋯
3,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,18.0,3.24,5.832,10.4976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,3.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,6.25,15.625,39.0625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22.0,4.84,10.648,23.4256,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
6,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.01,0.001,0.0001,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
7,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,42.0,17.64,74.088,311.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,17.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
8

In [15]:
basic_model = lm(@formula(lwage ~ sex+shs+hsg+scl+clg+mw+so+we+occ2+ind2), df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

lwage ~ 1 + sex + shs + hsg + scl + clg + mw + so + we + occ2 + ind2

Coefficients:
───────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       t  Pr(>|t|)   Lower 95%     Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)   3.83899     0.0811716   47.29    <1e-99   3.67986     3.99812
sex          -0.0649486   0.0152641   -4.25    <1e-04  -0.0948727  -0.0350244
shs          -0.53551     0.0512017  -10.46    <1e-24  -0.635887   -0.435132
hsg          -0.459379    0.0273044  -16.82    <1e-61  -0.512907   -0.40585
scl          -0.386357    0.0255448  -15.12    <1e-49  -0.436435   -0.336278
clg          -0.187943    0.0233293   -8.06    <1e-15  -0.233678   -0.142207
mw           -0.038303

In [17]:
flexible_model = lm(@formula(lwage ~ sex+ (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+mw+so+we+occ2+ind2)), df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

lwage ~ 1 + sex + exp1 + exp2 + exp3 + exp4 + shs + hsg + scl + clg + mw + so + we + occ2 + ind2 + exp1 & shs + exp1 & hsg + exp1 & scl + exp1 & clg + exp1 & mw + exp1 & so + exp1 & we + exp1 & occ2 + exp1 & ind2 + exp2 & shs + exp2 & hsg + exp2 & scl + exp2 & clg + exp2 & mw + exp2 & so + exp2 & we + exp2 & occ2 + exp2 & ind2 + exp3 & shs + exp3 & hsg + exp3 & scl + exp3 & clg + exp3 & mw + exp3 & so + exp3 & we + exp3 & occ2 + exp3 & ind2 + exp4 & shs + exp4 & hsg + exp4 & scl + exp4 & clg + exp4 & mw + exp4 & so + exp4 & we + exp4 & occ2 + exp4 & ind2

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                        Coef.  Std. Error      t  Pr(>|t|)    Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────

In [20]:
extra_flexible_model = lm(@formula(lwage ~ sex+ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+mw+so+we+occ2+ind2)*(exp1+exp2+exp3+exp4+shs+hsg+scl+clg+mw+so+we+occ2+ind2)), df)

BoundsError: BoundsError: attempt to access Tuple{} at index [1]

In [16]:
using DataFrames

# Crear un DataFrame de ejemplo
df = DataFrame(occ2 = [1, 2, 3, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12,13,14,15,16,17,18,19,20,21,22,15])

# Crear nuevas columnas occ2_1, occ2_2, ..., occ2_10
for i in 1:22
    col_name = Symbol("occ2_$i")
    df = transform(df, :occ2 => ByRow(x -> x == i ? 1 : 0) => col_name)
end

# Mostrar el DataFrame resultante
println(df)

26×23 DataFrame
 Row │ occ2   occ2_1  occ2_2  occ2_3  occ2_4  occ2_5  occ2_6  occ2_7  occ2_8  occ2_9  occ2_10  occ2_11  occ2_12  occ2_13  occ2_14  occ2_15  occ2_16  occ2_17  occ2_18  occ2_19  occ2_20  occ2_21  occ2_22 
     │ Int64  Int64   Int64   Int64   Int64   Int64   Int64   Int64   Int64   Int64   Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64    Int64   
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │     1       1       0       0       0       0       0       0       0       0        0        0        0        0        0        0        0        0        0        0        0        0        0
   2 │     2       0       1       0       0       0       0       0       0       0        0        0        0        0        0        0        0        0        0        

## 3. Linear Regressions